# Dependencies / Setup

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import json
import datetime

# Today's Date
now = datetime.datetime.now()

# PyMySQL
import pymysql
pymysql.install_as_MySQLdb()

# Import API key
from config import api_key

timetag = now.strftime("%Y_%m_%d_%H%M")
print(timetag)

2018_12_10_1117


## Import video games list from kaggle source

In [2]:
csv_file = "vgsales.csv"
vgsales_data_df = pd.read_csv(csv_file)
vgsales_data_df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
# Mask for specific rank (or # in list)
# I am tasked with pulling ranks 11001 to the end, 16600
# Was only able to pull the first 3000 entries on free api key (to 14001)
# Resumed the remaining entries on a second free api key, 14002 to 16600
start_rank = 14002
end_rank = 16600
mask = (vgsales_data_df['Rank'] >= start_rank) & (vgsales_data_df['Rank'] <= end_rank)

#ESRB, total_rating, popularity, hypes
names_list = vgsales_data_df.loc[mask]["Name"].values.tolist()
names_list

['Smashing Drive',
 'Age of Mythology: The Titans',
 'Fighting Vipers 2',
 'Teenage Mutant Ninja Turtles 3: Mutant Nightmare',
 'Sudoku Ball Detective',
 '12-Sai. Koisuru Diary',
 'Castlevania: Lords of Shadow - Reverie',
 'Napoleon Dynamite: The Game',
 'Beatdown: Fists of Vengeance',
 'Enchanted Arms (JP sales)',
 'Monster Force',
 'World of Goo',
 'Onsei Kanjou Sokuteiki: Kokoro Scan',
 'Clock Zero: Shuuen no Ichibyou Portable',
 'San Goku Shi DS 3',
 'Trick x Logic: Season 2',
 'Challenge Me: Word Puzzles',
 'Hakuouki: Zuisouroku',
 'World in Conflict',
 'World Championship Pool 2004',
 'Derby Tsuku 5: Derby Uma o Tsukurou!',
 'S.T.A.L.K.E.R.: Clear Sky',
 'Jikkyou Powerful Major League 2009',
 'SD Gundam Force: Showdown!',
 'Ray Gigant',
 'Negima!? 3-Jikanme ~Koi to Mahou to Sekaiju Densetsu~',
 'Flow: Urban Dance Uprising',
 'Ailu de Puzzle',
 'World in Conflict: Complete Edition',
 'True Pinball',
 "Jane's Hotel",
 'Stadium Games',
 'RLH: Run Like Hell',
 'Power Play Pool',
 'J-

In [4]:
print(api_key)

97766d9c0926b0c2bf5d862eaa2378b0


### Perform API Calls
* Perform an ESRB check on each name using a series of successive API calls.
* Include a print log of each video game as it is being processed (with the game name and esrb rating).


In [5]:
#API config data
# Save config information
url = "https://api-endpoint.igdb.com"
headers = {"user-key" : api_key,"Accept" : "application/json"}
# Build partial query URL
query_url = f"{url}/games/?search="

In [6]:
#ESRB, total_rating, popularity, hypes

# set up lists to hold reponse info
title = []
ESRB = []
total_rating = []
popularity = []
hypes = []

# Creates the count variable for record keeping
count = 1

# Loop through the list of cities and perform a request for data on each
print("Beginning Data Retrieval")
print("------------------------")
for name in names_list:
    try:
        print(f"Processing record {count} of {len(names_list)} | {name}")
        print(f"{query_url}{name}")
        response = requests.get(query_url + name+"&fields=name,esrb,total_rating,popularity", headers=headers).json()
        
        #Append information to lists
        try:
            title.append(response[0]["name"])
        except KeyError:
            title.append("NaN")
            continue
        try:
            ESRB.append(response[0]["esrb"])
        except KeyError:
            ESRB.append("NaN")
            continue
        try:
            total_rating.append(response[0]["total_rating"])
        except KeyError:
            total_rating.append("NaN")
            continue
        try:
            popularity.append(response[0]["popularity"])
        except KeyError:
            popularity.append("NaN")
            continue
        try:
            hypes.append(response[0]["hypes"])
        except KeyError:
            hypes.append("NaN")
    
    except:
        print("Game not found, skipped.")
        
    finally:
        if count < 6000:
            count += 1
        else:
            break

Beginning Data Retrieval
------------------------
Processing record 1 of 2598 | Smashing Drive
https://api-endpoint.igdb.com/games/?search=Smashing Drive
Processing record 2 of 2598 | Age of Mythology: The Titans
https://api-endpoint.igdb.com/games/?search=Age of Mythology: The Titans
Processing record 3 of 2598 | Fighting Vipers 2
https://api-endpoint.igdb.com/games/?search=Fighting Vipers 2
Processing record 4 of 2598 | Teenage Mutant Ninja Turtles 3: Mutant Nightmare
https://api-endpoint.igdb.com/games/?search=Teenage Mutant Ninja Turtles 3: Mutant Nightmare
Processing record 5 of 2598 | Sudoku Ball Detective
https://api-endpoint.igdb.com/games/?search=Sudoku Ball Detective
Processing record 6 of 2598 | 12-Sai. Koisuru Diary
https://api-endpoint.igdb.com/games/?search=12-Sai. Koisuru Diary
Processing record 7 of 2598 | Castlevania: Lords of Shadow - Reverie
https://api-endpoint.igdb.com/games/?search=Castlevania: Lords of Shadow - Reverie
Processing record 8 of 2598 | Napoleon Dynam

Processing record 67 of 2598 | The Ant Bully
https://api-endpoint.igdb.com/games/?search=The Ant Bully
Processing record 68 of 2598 | Sega Arcade Gallery
https://api-endpoint.igdb.com/games/?search=Sega Arcade Gallery
Processing record 69 of 2598 | Crazy Machines
https://api-endpoint.igdb.com/games/?search=Crazy Machines
Processing record 70 of 2598 | Astrology DS
https://api-endpoint.igdb.com/games/?search=Astrology DS
Processing record 71 of 2598 | My World, My Way (US sales)
https://api-endpoint.igdb.com/games/?search=My World, My Way (US sales)
Processing record 72 of 2598 | Planescape: Torment
https://api-endpoint.igdb.com/games/?search=Planescape: Torment
Processing record 73 of 2598 | Tao Adventure: Curse Demon
https://api-endpoint.igdb.com/games/?search=Tao Adventure: Curse Demon
Processing record 74 of 2598 | Aegis of Earth: Protonovus Assault
https://api-endpoint.igdb.com/games/?search=Aegis of Earth: Protonovus Assault
Processing record 75 of 2598 | Hello Kitty: Roller Rescu

Processing record 130 of 2598 | Zatch Bell! Electric Arena
https://api-endpoint.igdb.com/games/?search=Zatch Bell! Electric Arena
Processing record 131 of 2598 | Boulder Dash: Rocks!
https://api-endpoint.igdb.com/games/?search=Boulder Dash: Rocks!
Processing record 132 of 2598 | NHRA Drag Racing: Countdown to the Championship
https://api-endpoint.igdb.com/games/?search=NHRA Drag Racing: Countdown to the Championship
Processing record 133 of 2598 | The Void
https://api-endpoint.igdb.com/games/?search=The Void
Processing record 134 of 2598 | TOCA Race Driver 3
https://api-endpoint.igdb.com/games/?search=TOCA Race Driver 3
Processing record 135 of 2598 | Daito Giken Premium Pachi-Slot Collection: Yoshimune
https://api-endpoint.igdb.com/games/?search=Daito Giken Premium Pachi-Slot Collection: Yoshimune
Processing record 136 of 2598 | Maximum Racing: Drag & Stock Racer
https://api-endpoint.igdb.com/games/?search=Maximum Racing: Drag & Stock Racer
Processing record 137 of 2598 | Sky Dancers


Processing record 191 of 2598 | Batman Forever: The Arcade Game
https://api-endpoint.igdb.com/games/?search=Batman Forever: The Arcade Game
Processing record 192 of 2598 | Romance of the Three Kingdoms (3DS)
https://api-endpoint.igdb.com/games/?search=Romance of the Three Kingdoms (3DS)
Processing record 193 of 2598 | Electroplankton
https://api-endpoint.igdb.com/games/?search=Electroplankton
Processing record 194 of 2598 | Cabela's Outdoor Adventures
https://api-endpoint.igdb.com/games/?search=Cabela's Outdoor Adventures
Processing record 195 of 2598 | Makai Senki Disgaea 4: Fuuka & Desco-hen Hajime Mashita
https://api-endpoint.igdb.com/games/?search=Makai Senki Disgaea 4: Fuuka & Desco-hen Hajime Mashita
Processing record 196 of 2598 | Backyard NFL Football 2006
https://api-endpoint.igdb.com/games/?search=Backyard NFL Football 2006
Processing record 197 of 2598 | Game of Thrones (Telltale)
https://api-endpoint.igdb.com/games/?search=Game of Thrones (Telltale)
Processing record 198 of

Processing record 253 of 2598 | Jewel Master: Cradle of Rome
https://api-endpoint.igdb.com/games/?search=Jewel Master: Cradle of Rome
Processing record 254 of 2598 | Midnight Mysteries: The Edgar Allan Poe Conspiracy
https://api-endpoint.igdb.com/games/?search=Midnight Mysteries: The Edgar Allan Poe Conspiracy
Processing record 255 of 2598 | Beastly
https://api-endpoint.igdb.com/games/?search=Beastly
Processing record 256 of 2598 | Crash Time: Autobahn Pusuit
https://api-endpoint.igdb.com/games/?search=Crash Time: Autobahn Pusuit
Processing record 257 of 2598 | Earthworm Jim 2
https://api-endpoint.igdb.com/games/?search=Earthworm Jim 2
Processing record 258 of 2598 | Garfield: A Tale of Two Kitties
https://api-endpoint.igdb.com/games/?search=Garfield: A Tale of Two Kitties
Processing record 259 of 2598 | Tamagotchi no Doki Doki Dream Omisecchi
https://api-endpoint.igdb.com/games/?search=Tamagotchi no Doki Doki Dream Omisecchi
Processing record 260 of 2598 | Toushin Toshi
https://api-en

...

Processing record 2304 of 2598 | Durarara!! Relay
https://api-endpoint.igdb.com/games/?search=Durarara!! Relay
Processing record 2305 of 2598 | Moto Racer Advance
https://api-endpoint.igdb.com/games/?search=Moto Racer Advance
Processing record 2306 of 2598 | Duel Masters: Shadow of the Code
https://api-endpoint.igdb.com/games/?search=Duel Masters: Shadow of the Code
Processing record 2307 of 2598 | Full Auto 2: Battlelines
https://api-endpoint.igdb.com/games/?search=Full Auto 2: Battlelines
Processing record 2308 of 2598 | Freaky Flyers
https://api-endpoint.igdb.com/games/?search=Freaky Flyers
Processing record 2309 of 2598 | Army Men: Air Combat - The Elite Missions
https://api-endpoint.igdb.com/games/?search=Army Men: Air Combat - The Elite Missions
Processing record 2310 of 2598 | Rally Championship
https://api-endpoint.igdb.com/games/?search=Rally Championship
Processing record 2311 of 2598 | Happy Dance Collection
https://api-endpoint.igdb.com/games/?search=Happy Dance Collection


Processing record 2367 of 2598 | Hakuouki: Shinsengumi Kitan
https://api-endpoint.igdb.com/games/?search=Hakuouki: Shinsengumi Kitan
Processing record 2368 of 2598 | Dance with Devils
https://api-endpoint.igdb.com/games/?search=Dance with Devils
Processing record 2369 of 2598 | Wand of Fortune
https://api-endpoint.igdb.com/games/?search=Wand of Fortune
Processing record 2370 of 2598 | Arabians Lost: The Engagement on Desert
https://api-endpoint.igdb.com/games/?search=Arabians Lost: The Engagement on Desert
Processing record 2371 of 2598 | Gobliiins 4
https://api-endpoint.igdb.com/games/?search=Gobliiins 4
Processing record 2372 of 2598 | S.Y.K: Shinsetsu Saiyuuki Portable
https://api-endpoint.igdb.com/games/?search=S.Y.K: Shinsetsu Saiyuuki Portable
Processing record 2373 of 2598 | Crouching Tiger, Hidden Dragon
https://api-endpoint.igdb.com/games/?search=Crouching Tiger, Hidden Dragon
Processing record 2374 of 2598 | Meru Purana
https://api-endpoint.igdb.com/games/?search=Meru Purana


Processing record 2432 of 2598 | Kamigami no Asobi InFinite
https://api-endpoint.igdb.com/games/?search=Kamigami no Asobi InFinite
Processing record 2433 of 2598 | Shinseiki Evangelion: Koutetsu no Girlfriend 2nd Portable
https://api-endpoint.igdb.com/games/?search=Shinseiki Evangelion: Koutetsu no Girlfriend 2nd Portable
Processing record 2434 of 2598 | Clive Barker's Jericho
https://api-endpoint.igdb.com/games/?search=Clive Barker's Jericho
Processing record 2435 of 2598 | Tenjin Ranman: Happy GO Lucky!!
https://api-endpoint.igdb.com/games/?search=Tenjin Ranman: Happy GO Lucky!!
Processing record 2436 of 2598 | Getsuei Gakuen: kou
https://api-endpoint.igdb.com/games/?search=Getsuei Gakuen: kou
Processing record 2437 of 2598 | Worms 2
https://api-endpoint.igdb.com/games/?search=Worms 2
Processing record 2438 of 2598 | The Cages: Pro Style Batting Practice
https://api-endpoint.igdb.com/games/?search=The Cages: Pro Style Batting Practice
Processing record 2439 of 2598 | Brothers Conflic

Processing record 2492 of 2598 | Summer Athletics
https://api-endpoint.igdb.com/games/?search=Summer Athletics
Processing record 2493 of 2598 | Daito Giken Koushiki Pachi-Slot Simulator: 24 - Twenty-Four
https://api-endpoint.igdb.com/games/?search=Daito Giken Koushiki Pachi-Slot Simulator: 24 - Twenty-Four
Processing record 2494 of 2598 | The Smurfs
https://api-endpoint.igdb.com/games/?search=The Smurfs
Processing record 2495 of 2598 | Legends of Oz: Dorothy's Return
https://api-endpoint.igdb.com/games/?search=Legends of Oz: Dorothy's Return
Processing record 2496 of 2598 | Unending Bloody Call
https://api-endpoint.igdb.com/games/?search=Unending Bloody Call
Processing record 2497 of 2598 | Shutsugeki! Otometachi no Senjou 2: Ikusabana no Kizuna
https://api-endpoint.igdb.com/games/?search=Shutsugeki! Otometachi no Senjou 2: Ikusabana no Kizuna
Processing record 2498 of 2598 | Snood 2: On Vacation
https://api-endpoint.igdb.com/games/?search=Snood 2: On Vacation
Processing record 2499 of

Processing record 2554 of 2598 | Bound By Flame
https://api-endpoint.igdb.com/games/?search=Bound By Flame
Processing record 2555 of 2598 | Psychic Detective
https://api-endpoint.igdb.com/games/?search=Psychic Detective
Processing record 2556 of 2598 | Codename: Panzers Complete Collection
https://api-endpoint.igdb.com/games/?search=Codename: Panzers Complete Collection
Processing record 2557 of 2598 | Super Robot Taisen: Original Generation
https://api-endpoint.igdb.com/games/?search=Super Robot Taisen: Original Generation
Processing record 2558 of 2598 | Dramatical Murder Re:code
https://api-endpoint.igdb.com/games/?search=Dramatical Murder Re:code
Processing record 2559 of 2598 | Pro Evolution Soccer 2008
https://api-endpoint.igdb.com/games/?search=Pro Evolution Soccer 2008
Processing record 2560 of 2598 | Sugar + Spice! Anoko no Suteki na Nanimokamo
https://api-endpoint.igdb.com/games/?search=Sugar + Spice! Anoko no Suteki na Nanimokamo
Processing record 2561 of 2598 | Hanasaku Man

### Save Data

In [7]:
#Create a dictionary of lists
games = {"name":title,
         "esrb":ESRB,
         "popularity":popularity,
         "hypes":hypes,
         "total_rating":total_rating}
print(games)

{'name': ['Smashing Drive', 'Age of Mythology: The Titans', 'Fighting Vipers 2', 'Teenage Mutant Ninja Turtles 3: Mutant Nightmare', 'Sudoku Ball: Detective', '12-sai. Koi suru Diary', 'Castlevania: Lords of Shadow - Reverie', 'Napoleon Dynamite: The Game', 'Beat Down: Fists of Vengeance', 'Enchanted Arms', 'Monster Force', 'World of Goo', 'Chotto Sunao ni Donburi Kanjou', 'Clock Zero: Shuuen no Ichibyou', 'San Goku Shi', 'Trick x Logic Season 2', 'Challenge Me: Brain Puzzles', 'Hakuoki Zuisouroku', 'World in Conflict', 'World Championship Pool 2004', 'J-League Pro Soccer Club o Tsukurou! 5', 'S.T.A.L.K.E.R.: Clear Sky', 'Jikkyou Powerful Major League 2009', 'SD Gundam Force: Showdown!', 'Ray Gigant', 'Mahou Sensei Negima!? Neo-Pactio Fight!!', 'Flow: Urban Dance Uprising', 'Ailu de Puzzle!', 'World in Conflict: Complete Edition', 'True Pinball', "Jane's Hotel", 'Stadium Games', 'Run Like Hell!', 'Power Play Pool', 'Pro Striker 2', 'ObsCure', 'Caesar III', 'Super Robot Wars Scramble Co

In [8]:
#Create a Pandas DataFrame
# create dataframe oriented by index first since lists are not the same length
games_df = pd.DataFrame.from_dict(games, orient='index')
#Transpose the dataframe
games_df = games_df.transpose()
#Preview
games_df.head()

,name,esrb,popularity,hypes,total_rating
0,Smashing Drive,{'rating': 5},1.33333,NaN,35
1,Age of Mythology: The Titans,NaN,1.33333,NaN,42.5
2,Fighting Vipers 2,NaN,1,NaN,65
3,Teenage Mutant Ninja Turtles 3: Mutant Nightmare,{'rating': 5},1.33333,NaN,NaN
4,Sudoku Ball: Detective,{'rating': 4},1.33333,NaN,35


In [9]:
#Output DataFrame as a csv
games_df.to_csv("Output_MHC/games_14002_to_16600_df.csv", encoding = "utf-8", index = False)

In [10]:
#Split the esrb column
esrb_split_df = games_df['esrb'].apply(pd.Series)
esrb_split_df.head()

C:\Users\got_a\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
C:\Users\got_a\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)


,rating,0,synopsis
0,5.0,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,5.0,NaN,NaN
4,4.0,NaN,NaN


In [11]:
#Concat original df and split esrb df
split_games_df = pd.concat([games_df.drop(['esrb'], axis = 1), esrb_split_df],axis=1)
split_games_df.rename(columns = {'rating':'esrb'}, inplace = True)
split_games_df.drop(['hypes','synopsis',0], axis = 1, inplace = True)
split_games_df.head(100)

,name,popularity,total_rating,esrb
0,Smashing Drive,1.33333,35,5.0
1,Age of Mythology: The Titans,1.33333,42.5,NaN
2,Fighting Vipers 2,1,65,NaN
3,Teenage Mutant Ninja Turtles 3: Mutant Nightmare,1.33333,NaN,5.0
4,Sudoku Ball: Detective,1.33333,35,4.0
5,12-sai. Koi suru Diary,3,67.4621,NaN
6,Castlevania: Lords of Shadow - Reverie,1.33333,83.7349,NaN
7,Napoleon Dynamite: The Game,2.33333,49,4.0
8,Beat Down: Fists of Vengeance,1.33333,NaN,6.0
9,Enchanted Arms,2.66667,77.7321,5.0


In [12]:
split_games_df.dtypes

name             object
popularity       object
total_rating     object
esrb            float64
dtype: object

In [13]:
#Output Cleanee DataFrame as a csv
split_games_df.to_csv("Output_MHC/games_14002_to_16600_df_clean.csv", encoding = "utf-8", index = False)